In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: feature-test
Azure region: germanywestcentral
Subscription id: 779ddc6a-162c-43db-9f05-edaaa1cde3ba
Resource group: rg-preview-features


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute


compute_cluster_name = "cpu-compute"

try:
    compute_target = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, use this cluster that was found.")
except:
    print("Creating new cluster...")
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())

Found existing cluster, use this cluster that was found.

Running
{'errors': [], 'creationTime': '2023-03-17T14:47:19.901521+00:00', 'createdBy': {'userObjectId': 'ec782876-5d01-49ec-8043-380656d55163', 'userTenantId': 'ce849bab-cc1c-465b-b62e-18f07c9ac198', 'userName': None}, 'modifiedTime': '2023-03-19T17:43:00.377141+00:00', 'state': 'Running', 'vmSize': 'STANDARD_E4DS_V4'}


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C" : choice(0.01, 0.1, 1),
    "--max_iter" : choice(20, 40, 60, 100, 150, 200)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=5)


if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                            script='train.py',
                            compute_target="cpu-compute",
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                    hyperparameter_sampling=ps,
                    policy=policy,
                    primary_metric_name='Accuracy',
                    primary_metric_goal= PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=4,
                    max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_01bcd8ff-5796-4ebf-bedc-3c9a0e524237
Web View: https://ml.azure.com/runs/HD_01bcd8ff-5796-4ebf-bedc-3c9a0e524237?wsid=/subscriptions/779ddc6a-162c-43db-9f05-edaaa1cde3ba/resourcegroups/rg-preview-features/workspaces/feature-test&tid=ce849bab-cc1c-465b-b62e-18f07c9ac198

Streaming azureml-logs/hyperdrive.txt

[2023-04-09T22:49:25.420550][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-04-09T22:49:25.7584751Z][SCHEDULER][INFO]Scheduling job, id='HD_01bcd8ff-5796-4ebf-bedc-3c9a0e524237_0' 
[2023-04-09T22:49:25.8834291Z][SCHEDULER][INFO]Scheduling job, id='HD_01bcd8ff-5796-4ebf-bedc-3c9a0e524237_1' 
[2023-04-09T22:49:26.0068598Z][SCHEDULER][INFO]Scheduling job, id='HD_01bcd8ff-5796-4ebf-bedc-3c9a0e524237_2' 
[2023-04-09T22:49:26.132111][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-04-09T22:49:26.1749965Z][SCHEDULER][INFO]Scheduling job, id='HD_01bcd8ff-5796-4ebf-bedc-3c9a0e524237_3' 
[2023-

{'runId': 'HD_01bcd8ff-5796-4ebf-bedc-3c9a0e524237',
 'target': 'cpu-compute',
 'status': 'Completed',
 'startTimeUtc': '2023-04-09T22:49:24.639722Z',
 'endTimeUtc': '2023-04-09T22:52:26.924876Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e3d26362-2953-4aea-b18a-1d26f9376da2',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': '18',
  'score': '0.9112291350531108',
  'best_child_run_id': 'HD_01bcd8ff-5796-4ebf-bedc-3c9a0e524237_3',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_01bcd8ff-5796-4ebf-bedc-3c9a0e524237_3'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClient

In [22]:
import joblib
# Get best run and save the model from that run.


hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric()

print("best run details :", hyperdrive_best_run.get_details())
print('\n')
print("best run file names :", hyperdrive_best_run.get_file_names())
print('\n')
print("best run metrics :", hyperdrive_best_run.get_metrics())

model = hyperdrive_best_run.register_model(model_name='hyperdrive_log_reg_best_run', model_path='outputs/model.joblib', tags={"Method" : "HyperDrive"},properties={"Accuracy" : hyperdrive_best_run.get_metrics()["Accuracy"]})

best run details : {'runId': 'HD_f8e536dc-a633-48ae-a3f1-54baff7ea34f_3', 'target': 'cpu-compute', 'status': 'Completed', 'startTimeUtc': '2023-03-18T23:57:18.525338Z', 'endTimeUtc': '2023-03-18T23:58:56.508152Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlcdsi', 'ContentSnapshotId': 'ce452998-b92b-4414-9a0f-be9df2c6d51d', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.01', '--max_iter', '150'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'cpu-compute', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 'credentialPassthrough': False, 'identity': None, 'environment': {'name': 'sklearn-en

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])


In [7]:
import pandas as pd
from train import clean_data
from sklearn.model_selection import train_test_split
from azureml.core import Dataset

# Use the clean_data function to clean your data.
x, y = clean_data(ds)


all_data = pd.concat([x,y], axis=1)

train_data, test_data = train_test_split(all_data, test_size=0.2, random_state=49)

train_data.to_csv('training/training_df.csv')

ds1= ws.get_default_datastore()
ds1.upload(src_dir='training', target_path='training_data')

train_ds = Dataset.Tabular.from_delimited_files(path=[(ds1, ('training_data/training_df.csv'))])

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 1 files
Target already exists. Skipping upload for training_data/training_df.csv
Uploaded 0 files


In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_ds,
    label_column_name='y',
    n_cross_validations=5,
    compute_target="cpu-compute",
    enable_early_stopping = True,
    enable_onnx_compatible_models = True)

In [11]:
# Submit your automl run

remote_run = exp.submit(config=automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on cpu-compute with default configuration
Running on remote compute: cpu-compute


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_7be08b18-e6fa-496b-8b0b-6e71f5433c62,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+========================

In [12]:
# Retrieve and save your best automl model.

remote_run.get_metrics()
remote_run.get_tags()

{'model_explain_run': 'best_run',
 '_aml_system_automl_run_workspace_id': 'b83f7342-22dd-41c0-9a4c-6fe3c2bcc0d6',
 '_aml_system_azureml.automlComponent': 'AutoML',
 'pipeline_id_000': '5dfac790c5c209f98a1da2dc1c7fb76f0397324f;c7af0367625be6ac5c2fecbfc72ed444cb7a2111;799d2168db11fc19b9e1c6c1df62f8981ad39fe9;44b5d2d23120dfcf96a7dc6389a293d17a966b1b;3735210984ea10097d5c91905cf6300ac278cd89;5304e705d6f401cb07f85bcf0ca59ac1f84dfcf5;294650ec23bc3d067a4972de7ceb12f6e97260fa;3caa818370e5e0c328d6266c359019bd31681270;79ebb3b8c385e3190e3dd4c1fa28b74730528045;589334a57221e0720a95cf217c065115a7b841d1;5a8eb9d9f67b1dd7e9e4062585daa2417ea78989;35b8e2db3c5f6e714072e1be9f0210fe48426975;da45f657e4591caadf1589a85a1f7b8932585595;797e3e972ad02d947be4c64d2b06b775b6df7a49;3fd2e115482e0c7b92aa3297688aefb6c3aa6fb8;1dfdf370ba770047b14bfa59c40920815defcdab;5235e3d56688f5f38ef82d73fba7c2e698fc1af2;17f87d449bb43aaf3cdae4b7eb6dd6e7fbdaf343;dc68fbd594530b5d52812dd9b3ffa3d6d8ba625e;2032292dd602836ed754a266c771ac100c4c

In [13]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

automl_best_run_onnx, automl_fitted_model_onnx = remote_run.get_output(return_onnx_model=True)

OnnxConverter.save_onnx_model(automl_fitted_model_onnx, './outputs/best_automl_model.onnx' )